In [2]:
!pip install psycopg2

In [4]:
!pip install include

In [115]:
def open_connection():
    global conn_mysql
    global conn_psql
    try:
        conn_mysql= msql.connect(host='localhost', database='davis2', user='root', password='bo2108luwatife@1')
        print('Connected to MySQL Database')
    except Error as e:
        print("Error while connecting to MySQL", e)
        sys.exit(1)
    #Postgresql connection
    try:
        conn_psql = psycopg2.connect(host='localhost', database='sensor_data', user='postgres', password='bo2108luwatife@1')
        print('Connected to PostgreSQL Database')
    except psycopg2.Error as e:
        print('PSQL: Unable to connect!\n{0}').format(e)
        sys.exit(1)

In [116]:
open_connection()

Connected to MySQL Database
Connected to PostgreSQL Database


In [117]:
def select_from_sql(database, table_name):
    #create table
    mysql_query = pd.read_sql_query(f'''SELECT flow1, occupancy1, flow2, 
                    flow3, occupancy3 FROM {database}.{table_name}''', conn_mysql)
    df = pd.DataFrame(mysql_query)
    return df

In [118]:
sql_data = select_from_sql('davis2', 'load_data')

In [119]:
def migrate_to_psql(filename, schema_name, table_name):
     # Cursors initializations
    cur_mysql = conn_mysql.cursor(dictionary=True)
    cur_psql = conn_psql.cursor()

    #create schema and table in Postgres
    create_psql_schema = f'CREATE SCHEMA IF NOT EXISTS {schema_name}'
    create_psql_table = f'''CREATE TABLE {schema_name}.{table_name} (
                            flow1 FLOAT, occupancy1 FLOAT, flow2 FLOAT, flow3 FLOAT, occupancy3 FLOAT)'''
    cur_psql.execute(create_psql_schema)
    print('Schema is created')
    cur_psql.execute(create_psql_table)
    print('Table is created')

    for i, row in new_df.iterrows():
        try:
            psql = f'''INSERT INTO {schema_name}.{table_name} (flow1, occupancy1, flow2, flow3, occupancy3)
                    VALUES (%s, %s, %s, %s, %s)'''
            cur_psql.execute(psql, tuple(row))
            print('Data Inserted!')

        except psycopg2.Error as e:
            print("cannot execute that query!", e.pgerror)
            sys.exit("Some problem occured with that query!")   
    ## Closing cursors
    cur_mysql.close()
    cur_psql.close()

    ## Committing 
    conn_psql.commit()

    ## Closing database connections
    conn_mysql.close()
    conn_psql.close()

In [120]:
migrate_to_psql(sql_data, 'a','b')

Schema is created
Table is created
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data 

Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inser

Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inser

Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inser

Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inser

...

Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inser

Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inser

Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inser

Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inser

Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inserted!
Data Inser

In [ ]:
# Step 3: Define the DAG, i.e. the workflow
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
# DAG's arguments
default_args = {
        'owner': 'Mubarak',
        'start_date':dt.datetime(2021, 22, 9, 9, 0, 0),
        'concurrency': 1,
        'retries': 0
        }
# DAG's operators, or bones of the workflow
with DAG('Database Migration',
        catchup=False, # To skip any intervals we didn't run
        default_args=default_args,
        schedule_interval='* 1 * * * *', # 's m h d mo y'; set to run every minute.
        ) as dag:

    load_csv = PythonOperator(
            task_id='Open Connection',
            python_callable=open_connection,
            )

    upload_csv_to_sql = PythonOperator(
            task_id='MySQL Database to CSV',
            python_callable=to_sql
            )
    
    upload_csv_to_sql = PythonOperator(
            task_id='CSV to Database',
            python_callable=migrate_to_psql
            )

# The actual workflow
load_csv >> upload_csv_to_sql >> migrate_to_psql